In [1]:
%%capture
from numpy import int16
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime
from threading import Timer
from pandas import read_excel
from siuba import _

from oactf import actif, patient, driver, SDATA

date_du_jour = datetime.today().strftime("%d_%m_%Y")


In [2]:
""" def the_upload():
    year = ['2022-2023','2023-2024']
    schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
    schooling_dreams = schooling_dreams[schooling_dreams['schooling_year'].isin(year)]
    schooling_dreams = schooling_dreams[schooling_dreams.closed == False]
    payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]
    return schooling_dreams, payed_schooling_dreams

t = Timer(3000.0,the_upload)
t.start()
schooling_dreams,payed_schooling_dreams = the_upload() """

from time import sleep

sleep(3)

year = ['2022-2023','2023-2024']
schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
schooling_dreams = schooling_dreams[schooling_dreams['schooling_year'].isin(year)]
schooling_dreams = schooling_dreams[schooling_dreams.closed == False]
payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]


' def the_upload():\n    year = [\'2022-2023\',\'2023-2024\']\n    schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime(\'%Y-%m-%d\'))}.xlsx")\n    schooling_dreams = schooling_dreams[schooling_dreams[\'schooling_year\'].isin(year)]\n    schooling_dreams = schooling_dreams[schooling_dreams.closed == False]\n    payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]\n    return schooling_dreams, payed_schooling_dreams\n\nt = Timer(3000.0,the_upload)\nt.start()\nschooling_dreams,payed_schooling_dreams = the_upload() '

In [3]:
schooling_dreams.closed.unique()
schooling_dreams.eskew_peye.unique()
payed_schooling_dreams.eskew_peye.unique()


array([False])

array(['1', '---'], dtype=object)

array(['1'], dtype=object)

In [4]:
actif.id_patient.count()
patient.id_patient.count()
schooling_dreams.caseid.count()
payed_schooling_dreams.caseid.count()


9770

180007

533

464

In [5]:
actif.to_excel('actif.xlsx',index=False)


In [6]:
comp = actif[actif.complete_at_least=="yes"]
comp.id_patient.count()
comp.test_results.fillna("0,",inplace=True)
comp.test_results.value_counts()


9651

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_29668/1809875057.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp.test_results.fillna("0,",inplace=True)


0,       9172
0,,2,     256
2,        218
0,,3,       2
1,          1
3,          1
2,,3,       1
Name: test_results, dtype: int64

In [7]:
ovc_data= comp
neg_data = comp[
    (comp.test_results!="0,,3,")&
    (comp.test_results!="0,,1,,2,")&
    (comp.test_results!="0,,2,,3,")&
    (comp.test_results!="3,")&
    (comp.test_results!="2,,3,")
]
ovc_data.id_patient.count()
neg_data.id_patient.count()
neg_data.test_results.value_counts()


9651

9647

0,       9172
0,,2,     256
2,        218
1,          1
Name: test_results, dtype: int64

In [8]:
positive_dreams= ovc_data[
    (ovc_data.test_results=="1,")|
    (ovc_data.test_results=="0,,1,")
]
positive_dreams.id_patient.count()


1

In [9]:
positive_dreams


,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ2,isEnrolledQ4
515,175251.0,NaN,Ouest,Port-au-Prince,21,0,yes,yes,1,1,...,primary,no,no,secondary,no,no,no,yes,yes,no


In [10]:
ovc_data.ovc_age.unique()
ovc_data.parenting.unique()
ovc_data.parenting_detailed.unique()


array(['18-24', '15-17', '25-29', '10-14', 'not_valid_age'], dtype=object)

array(['no', 'yes'], dtype=object)

array(['no', 'yes', 'has_started'], dtype=object)

In [11]:
ovcS_dreams = ovc_data[
    (ovc_data.ovc_age=="10-14")|
    (ovc_data.ovc_age=="15-17")
]
ovcS_dreams['Gender'] = "Female"
ovcS_dreams.id_patient.count()


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_29668/1271102160.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovcS_dreams['Gender'] = "Female"


6222

In [12]:
ovcS_dreams = patient.merge(ovcS_dreams,on="id_patient",how="right")


In [13]:
ovcS_dreams[ovcS_dreams.dreams_code.isin(payed_schooling_dreams.dreams_code)].id_patient.count()
ovcS_dreams = ovcS_dreams[~ovcS_dreams.dreams_code.isin(payed_schooling_dreams.dreams_code)]
ovcS_dreams.id_patient.count()


328

5894

In [14]:
ovcS_dreams.parenting_detailed.unique()


array(['no', 'yes', 'has_started'], dtype=object)

In [15]:
datim_ovc_dreams = ovcS_dreams.pivot_table(index="commune",values="id_patient",columns=["Gender","ovc_age"],aggfunc='count',fill_value=0)
datim_ovc_dreams


Gender                         Female      
ovc_age                         10-14 15-17
commune                                    
Cap-Haïtien                       158   138
Carrefour                         241   165
Delmas                             66    67
Desdunes                           45    30
Dessalines                        323   113
Grande Saline                     111    79
Gressier                          149   122
Kenscoff                          933   322
La Chapelle                        99   100
Liancourt                          17    32
Limonade                           38    19
Milot                              13    16
Montrouis                          70    57
Petite Rivière de l'Artibonite     44    24
Plaine-du-Nord                     29    17
Port-au-Prince                    125   103
Pétionville                        67   101
Quartier-Morin                     25     9
Saint-Marc                        523   548
Tabarre                            89    43
Verrettes                         369   255

In [16]:
datim_ovc_dreams[('Female','<1')] = 0
datim_ovc_dreams[('Female','1-4')] = 0
datim_ovc_dreams[('Female','5-9')] = 0
datim_ovc_dreams[('Female','18+')] = 0
datim_ovc_dreams[('Male','<1')] = 0
datim_ovc_dreams[('Male','1-4')] = 0
datim_ovc_dreams[('Male','5-9')] = 0
datim_ovc_dreams[('Male','10-14')] = 0
datim_ovc_dreams[('Male','15-17')] = 0
datim_ovc_dreams[('Male','18+')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim_ovc_dreams = datim_ovc_dreams.reindex(columns= columns_orientation)
datim_ovc_dreams


Gender                         Female                         Male          \
ovc_age                            <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                      
Cap-Haïtien                         0   0   0   158   138   0    0   0   0   
Carrefour                           0   0   0   241   165   0    0   0   0   
Delmas                              0   0   0    66    67   0    0   0   0   
Desdunes                            0   0   0    45    30   0    0   0   0   
Dessalines                          0   0   0   323   113   0    0   0   0   
Grande Saline                       0   0   0   111    79   0    0   0   0   
Gressier                            0   0   0   149   122   0    0   0   0   
Kenscoff                            0   0   0   933   322   0    0   0   0   
La Chapelle                         0   0   0    99   100   0    0   0   0   
Liancourt                           0   0   0    17    32   0    0   0   0   
Limonade                            0   0   0    38    19   0    0   0   0   
Milot                               0   0   0    13    16   0    0   0   0   
Montrouis                           0   0   0    70    57   0    0   0   0   
Petite Rivière de l'Artibonite      0   0   0    44    24   0    0   0   0   
Plaine-du-Nord                      0   0   0    29    17   0    0   0   0   
Port-au-Prince                      0   0   0   125   103   0    0   0   0   
Pétionville                         0   0   0    67   101   0    0   0   0   
Quartier-Morin                      0   0   0    25     9   0    0   0   0   
Saint-Marc                          0   0   0   523   548   0    0   0   0   
Tabarre                             0   0   0    89    43   0    0   0   0   
Verrettes                           0   0   0   369   255   0    0   0   0   

Gender                                          
ovc_age                        10-14 15-17 18+  
commune                                         
Cap-Haïtien                        0     0   0  
Carrefour                          0     0   0  
Delmas                             0     0   0  
Desdunes                           0     0   0  
Dessalines                         0     0   0  
Grande Saline                      0     0   0  
Gressier                           0     0   0  
Kenscoff                           0     0   0  
La Chapelle                        0     0   0  
Liancourt                          0     0   0  
Limonade                           0     0   0  
Milot                              0     0   0  
Montrouis                          0     0   0  
Petite Rivière de l'Artibonite     0     0   0  
Plaine-du-Nord                     0     0   0  
Port-au-Prince                     0     0   0  
Pétionville                        0     0   0  
Quartier-Morin                     0     0   0  
Saint-Marc                         0     0   0  
Tabarre                            0     0   0  
Verrettes                          0     0   0

In [22]:


ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data.xlsx",na_rep="",index=False)
positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams.xlsx",na_rep="",index=False)
datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams.xlsx",na_rep="",index=True)

ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data_{date_du_jour}.xlsx",na_rep="",index=False)
positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams_{date_du_jour}.xlsx",na_rep="",index=False)
datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams_{date_du_jour}.xlsx",na_rep="",index=True)



# export positive ID

In [18]:
from siuba import _


In [19]:
export_positive = SDATA[_.id_patient.isin(positive_dreams.id_patient)]
export_positive.shape[0]


1

In [20]:
export_positive.to_excel(f"./ovc_results/positive_data/export_positive.xlsx",index=False)
#export_positive.to_excel(f"./ovc_results/positive_data/export_positive_{date_du_jour}.xlsx",index=False)


In [21]:
driver.quit()
